In [52]:
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.cross_validation import KFold
from utils.metrics import ndcg_scorer
from sklearn.cluster import KMeans
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier

path = '../data/processed/'
train_users = pd.read_csv(path + 'processed_train_users.csv', nrows=20000)
train_users.fillna(-1, inplace=True)
y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id'], axis=1, inplace=True)

x_train = train_users.values
label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)
kf = KFold(len(x_train), n_folds=10, random_state=42)

In [53]:
%time score = cross_val_score(xgb, x_train, encoded_y_train, cv=kf, scoring=ndcg_scorer)
print score.mean()

CPU times: user 2min 32s, sys: 580 ms, total: 2min 33s
Wall time: 42.2 s
0.776040731894


In [ ]:
test = pd.DataFrame()

xgb = XGBClassifier(nthread=-1, seed=42)
test['preds_xgb'] = cross_val_predict(xgb, x_train, encoded_y_train, cv=kf)

# m = KMeans(4, n_jobs=-1, random_state=42)
# cluster = m.fit_predict(train_users)
# test['cluster'] = cluster

xtree = ExtraTreesClassifier(n_jobs=-1)
test['preds_xtree'] = cross_val_predict(xgb, x_train, encoded_y_train, cv=kf)

xtree = RandomForestClassifier(n_jobs=-1)
test['preds_rf'] = cross_val_predict(xtree, x_train, encoded_y_train, cv=kf)

xtree = PassiveAggressiveClassifier(n_jobs=-1)
test['preds_pa'] = cross_val_predict(xtree, x_train, encoded_y_train, cv=kf)

In [ ]:
xgb = XGBClassifier(n_estimators=5, max_depth=5, nthread=-1, seed=42)
%time score = cross_val_score(xgb, test, encoded_y_train, cv=kf, scoring=ndcg_scorer)
print score.mean()

In [ ]:
b = BaggingClassifier(xgb, n_jobs=-1, warm_start=True)
b.fit(x_train[:-2000], encoded_y_train[:-2000])
b.